### Quickstart to the BPNet Pipeline Command
Author: Jacob Scheiber

The easiest way for most people to get started and integrate BPNet into their pipelines is through the command-line tools that come packaged with the bpnet-lite repository. These tools cover the common steps of training and using BPNet models and aim to be as interoperable as possible with other tools and pipelines by relying on common file formats for data inputs and outputs. Further, because these tools are implemented using the Python API, the models and files that result from these tools can be easily loaded into Python if you want to do subsequent analyses or extend them to build your own pipelines.

Here is a complete list of the existing individual command-line tools:

`negatives` - Take in a set of loci and a genome and output a set of loci that are as GC matched as possible. <br>
`fit` - Train a BPNet model and evaluate it. <br>
`evaluate` - Evaluate the performance of a BPNet model and output the statistics in a TSV. <br>
`attribute` - Apply a trained BPNet model to a set of loci and return the DeepLIFT/SHAP attributions. <br>
`seqlets` - Calculate attributions for a set of regions and then use the tangermeme recursive seqlet caller on them and return a BED file. <br>
`marginalize` - Calculate marginalization predictions and (optionally) attributions for a set of motifs in a database. <br>

Tying these steps together is the `pipeline` command, which goes all the raw data to BPNet analysis results. This involves converting BAM/SAM/bed/tsv files to bigWigs (if necessary), calling peaks using MACS3 (if necessary), calling GC-matched negatives to the peaks (if necessary), training and evaluating the BPNet model, calculating DeepLIFT/SHAP attributions, calling and annotating seqlets, running TF-MoDISCo, and running in silico marginalizations.

If you already have a model, you can skip the first few steps and go straight into the attributions and discovery steps.

#### Using the Pipeline

The pipeline is meant to be as simple and flexible as possible. Because it is a command-line tool, we will run our examples using `os.system`, but in practice you should use the command-line directly and not a Jupyter notebook / Python script.

##### Creating a BPNet Pipeline JSON

The pipeline is composed of many steps, and each of these steps have hyperparameters that can be set and require pointers to where the data are. To manage this process, each step has a JSON containing this information, and the pipeline has an umbrella JSON containing JSONs for each of these steps. As steps in the pipeline are executed (such as data processing or peak/negative calling), the pointers are updated in the subsequent steps and JSONs are written out so that you have a record of the exact commands being run in each step.

Having an umbrella JSON like this might seem overwhelming at first, but you can use the `pipeline-json` command to create a working JSON with default parameters for each of the steps. You should pass in the pointers to your data when running `pipeline-json` and it will automatically figure out if it needs to preprocess the data and do peak/negative calling.

Here is an example of how to create a pipeline JSON for a MYC BPNet model. Note that the data pointers are to remote files on the ENCODE Portal to demonstrate those capabilities, but they can also be filepaths on your local computer. All you need to do is pass in where your genome FASTA file is, where the input signal files are, where the control files are (if needed), the name you want all of the intermediary steps to use when saving results, and the name of the pipeline JSON.

In [1]:
import os

# https://jaspar.elixir.no/download/data/2026/CORE/JASPAR2026_CORE_vertebrates_non-redundant_pfms_meme.txt

#-m $HOME/common/JASPAR2026_CORE_vertebrates_non-redundant_pfms.meme

os.system("""
bpnet pipeline-json\
    -s $HOME/common/hg38.fa\
    -p https://www.encodeproject.org/files/ENCFF331IUK/@@download/ENCFF331IUK.bed.gz\
    -i https://www.encodeproject.org/files/ENCFF074GYD/@@download/ENCFF074GYD.bam\
    -i https://www.encodeproject.org/files/ENCFF122BQB/@@download/ENCFF122BQB.bam\
    -c https://www.encodeproject.org/files/ENCFF772ZHK/@@download/ENCFF772ZHK.bam\
    -n quick-myc\
    -o quick-myc-pipeline.json
""")


0

We can then take a look at the JSON. It is large, but worth looking at what each of the parameters is.

In [2]:
import json

with open("quick-myc-pipeline.json", "r") as infile:
    parameters = json.load(infile)
    
    print(json.dumps(parameters, indent=4))

{
    "in_window": 2114,
    "out_window": 1000,
    "name": "quick-myc",
    "model": null,
    "dtype": "float32",
    "device": "cuda",
    "batch_size": 64,
    "verbose": true,
    "min_counts": 0,
    "max_counts": 99999999,
    "random_state": null,
    "exclusion_lists": null,
    "sequences": "/home/jmschrei/common/hg38.fa",
    "loci": [
        "https://www.encodeproject.org/files/ENCFF331IUK/@@download/ENCFF331IUK.bed.gz"
    ],
    "signals": [
        "https://www.encodeproject.org/files/ENCFF074GYD/@@download/ENCFF074GYD.bam",
        "https://www.encodeproject.org/files/ENCFF122BQB/@@download/ENCFF122BQB.bam"
    ],
    "controls": [
        "https://www.encodeproject.org/files/ENCFF772ZHK/@@download/ENCFF772ZHK.bam"
    ],
    "skip": false,
    "dry_run": false,
    "preprocessing_parameters": {
        "unstranded": false,
        "fragments": false,
        "paired_end": false,
        "pos_shift": 0,
        "neg_shift": 0,
        "callpeaks_format": null,
       

The defaults for each of the steps assume that you want to train a BPNet model. This means that the negative sampling ratio is 0.33 (not 0.1 like for ChromBPNet) and that the `max_jitter` is 128 instead of 500 for ChromBPNet. If you are trying to train an accessibility model, or train a BPNet model on ATAC-/DNase-seq data directly, you may need to change those parameters manually after creating the JSON.

An important note about the structure of the JSON is that each of the steps will first look for the relevant parameters in their respective JSON (such as the "attribute_parameters" JSON) but, if the parameter is not set there, will inherit the parameter from the umbrella pipeline JSON. This means that you only have to modify the constant parameters, such as filenames to the data or the receptive field of the model, at the highest level of the JSON for the changes to take place in each of the steps. However, it also gives you the flexibility to have custom parameters for any individual step; remember, it only inherits from the umbrella JSON if it is not set, and it allows you to set different values from the umbrella JSON if you would like. This allows you to do things such as train a model on the human genome and evaluate on the mouse genome, or use bfloat16 to train the model and then float32 for the attributions, or train on promoter regions and evaluate at distal peaks.

##### Providing a motif database

You are not required to provide a motif database to the pipeline. However, if you do, the TF-MoDISCo report will include annotations for each of the discovered patterns and you will run in silico marginalizations using each of the motifs in the database.

In [3]:
os.system("""
bpnet pipeline-json\
    -s $HOME/common/hg38.fa\
    -p https://www.encodeproject.org/files/ENCFF331IUK/@@download/ENCFF331IUK.bed.gz\
    -i https://www.encodeproject.org/files/ENCFF074GYD/@@download/ENCFF074GYD.bam\
    -i https://www.encodeproject.org/files/ENCFF122BQB/@@download/ENCFF122BQB.bam\
    -c https://www.encodeproject.org/files/ENCFF772ZHK/@@download/ENCFF772ZHK.bam\
    -n test\
    -o test.json\
    -m $HOME/common/JASPAR2026_CORE_vertebrates_non-redundant_pfms.meme
""")

0

##### On ATAC-seq data

The pipeline above works well for TF ChIP-seq experiments but several changes are needed for ATAC-/DNase-seq experiments. First, the data usually come in the form of fragments where both the start and the end correspond to cuts. This can be specified using the `-f` flag. Second, the data is unstranded and so does not need to be split into `+` and `-` bigWigs but should be in just one. This can be specified using the `-u` flag. The data is paired end, and so the `-pe` flag should be specified if you need MACS3 to run peak calling on your data (not needed if you already have epaks). Finally, if you want to shift the position of the reads you can pass in a shift on the positive and negative strands using `-ps <int>` and `-ns <int>`.

In [4]:
os.system("""
bpnet pipeline-json\
    -s $HOME/common/hg38.fa\
    -p https://www.encodeproject.org/files/ENCFF748UZH/@@download/ENCFF748UZH.bed.gz\
    -i https://www.encodeproject.org/files/ENCFF337YBN/@@download/ENCFF337YBN.bam\
    -i https://www.encodeproject.org/files/ENCFF981FXV/@@download/ENCFF981FXV.bam\
    -i https://www.encodeproject.org/files/ENCFF144GBU/@@download/ENCFF144GBU.bam\
    -n example\
    -o example-pipeline.json\
    -m $HOME/common/JASPAR2026_CORE_vertebrates_non-redundant_pfms.meme\
    -u -f -pe -ps 4 -ns -4
""")

0

#### Running the Pipeline

Now that we have a JSON, running the pipeline is just as simple as `bpnet pipeline -p <fname>.json` where you plug in the pipeline parameter JSON you just created. In principle, this could be combined into one step where you produce the JSON and immediately run it. However, this is intentionally separated so that you can inspect and modify the JSON from the default parameters without interrupting the workflow. For example, if you wanted to run a larger model you would need to go in and manually change the number of filters or layers. 

(Apologies for the many lines of BAM processing output. This only happens when processing many BAMs and when running the command through Jupyter notebook like this. Given that, I didn't feel a need to fix the issue.)

In [5]:
!bpnet pipeline -p quick-myc-pipeline.json

Step 0.2: Convert data to bigWigs
chrEBV encountered in input but not in FASTA/chrom sizes.
ENCFF074GYD.bam: 15759996it [00:17, 913672.45it/s]

ENCFF122BQB.bam: 0it [00:00, ?it/s]
ENCFF122BQB.bam: 10064it [00:00, 100536.79it/s]
ENCFF122BQB.bam: 24436it [00:00, 125925.72it/s]
ENCFF122BQB.bam: 37030it [00:00, 78154.99it/s] 
ENCFF122BQB.bam: 89065it [00:00, 191890.46it/s]
ENCFF122BQB.bam: 111165it [00:00, 163297.67it/s]
ENCFF122BQB.bam: 136314it [00:00, 168631.36it/s]
ENCFF122BQB.bam: 154683it [00:00, 168425.69it/s]
ENCFF122BQB.bam: 172534it [00:01, 150130.14it/s]
ENCFF122BQB.bam: 188369it [00:01, 137594.98it/s]
ENCFF122BQB.bam: 204609it [00:01, 143474.20it/s]
ENCFF122BQB.bam: 219526it [00:01, 131796.19it/s]
ENCFF122BQB.bam: 235272it [00:01, 138143.63it/s]
ENCFF122BQB.bam: 249831it [00:01, 130254.77it/s]
ENCFF122BQB.bam: 263780it [00:01, 132485.63it/s]
ENCFF122BQB.bam: 280777it [00:01, 142557.40it/s]
ENCFF122BQB.bam: 295350it [00:02, 131941.84it/s]
ENCFF122BQB.bam: 311051it [00:02, 138482

At the end of this process, we have mapped the BAMs into bigWigs, identified GC-matched negatives from the provided peaks , trained and evaluated a BPNet model, run DeepLIFT/SHAP to get attributions, identified seqlets, run TF-MoDISco, and generated the report. All of these products are now on disk with a prefix of the name you provided in the `pipeline-json` command. This means that you can use the processed data in the next round if you would like, you can load the BPNet model into Python scripts and use it however you would like, and you can just look through the TF-MoDISco report without needing to run the command yourself. 

If you provided a motif database, there would be a Step 5.1/2 which involves running in silico marginalizations and creating a similar report based on the marginal influence of running each motif through the model.

You will also note several JSONs are now in the specified directories. These are the JSONs that were passed into the other subcommands, e.g. `bpnet fit` and `bpnet attribute`. This serves as a record of the precise commands used in each of those steps.

Finally, the `pipeline` command has no memory. This means that if you run it to process BAMs into bigWigs and then run it a second time it will reprocess the BAMs into bigWigs again. This is useful if you want to try out several different settings without being worried about previous runs contaminating your future runs, but can be annoying if you're only trying out different model architecture parameters. Fortunately, because bam2bw is so fast, it is not that big a deal to have to reprocess small to moderate sized datasets each time. If it is annoying, you can replace the BAM files with the bigWig files after the first run.